In [2]:
import pandas as pd

In [22]:
df = pd.read_csv('vistaalegre.csv', index_col = 0)
df.shape

(2500, 42)

In [23]:
df = df[['price', 'propertyType', 'size', 'exterior', 'rooms',
         'bathrooms', 'address', 'province', 'municipality',
         'latitude', 'longitude', 'status', 'newDevelopment', 'hasLift',
         'parkingSpace', 'priceByArea', 'detailedType', 'highlight']]

In [24]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,status,newDevelopment,hasLift,parkingSpace,priceByArea,detailedType,highlight
0,269000.0,flat,89.0,True,2,2,Calle Halcon,Madrid,Madrid,40.386730,-3.744151,good,False,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",3022.0,{'typology': 'flat'},NaN
1,180000.0,flat,63.0,True,3,1,Calle Carlos Paíno,Madrid,Madrid,40.386513,-3.745991,good,False,False,NaN,2857.0,{'typology': 'flat'},NaN
2,267000.0,flat,84.0,True,3,2,Calle del Espinar,Madrid,Madrid,40.386024,-3.743575,good,False,True,NaN,3179.0,{'typology': 'flat'},NaN
3,104500.0,flat,58.0,NaN,2,1,barrio Vista Alegre,Madrid,Madrid,40.386165,-3.743411,good,False,False,NaN,1802.0,{'typology': 'flat'},NaN
4,240000.0,flat,91.0,True,3,2,Calle de Antolina Merino s/n,Madrid,Madrid,40.386396,-3.743205,good,False,True,NaN,2637.0,{'typology': 'flat'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,199000.0,flat,47.0,False,1,1,"Calle Lavapiés, 52",Madrid,Madrid,40.409122,-3.701889,renew,False,False,NaN,4234.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}
2496,199000.0,flat,47.0,False,1,1,"Calle Lavapiés, 52",Madrid,Madrid,40.409122,-3.701889,renew,False,False,NaN,4234.0,{'typology': 'flat'},NaN
2497,160000.0,flat,83.0,True,2,1,Calle santa lucia,Madrid,Leganés,40.357973,-3.781615,good,False,False,NaN,1928.0,{'typology': 'flat'},NaN
2498,360000.0,flat,51.0,True,1,1,barrio Sol,Madrid,Madrid,40.414290,-3.707505,good,False,False,NaN,7059.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}


In [25]:
# Exterior cleaned
df['exterior'] = df['exterior'].fillna(False)

In [26]:
# hasLift cleaned
df['hasLift'] = df['hasLift'].fillna(False)

In [27]:
# highlight cleaned
df['highlight'] = df['highlight'].fillna("{'groupDescription': None}")

In [28]:
# parkingSpace cleaned
df['parkingSpace'] = df['parkingSpace'].fillna("{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}")

# Feature engineering

In [29]:
# parkingSpace included fixed
df['parkingSpace'] = df['parkingSpace'].apply(lambda x: "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}" if x == "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}" else x)

In [30]:
# unique parkingSpaces are now consistent
df['parkingSpace'].unique()

array(["{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 16000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 15000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 19500.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 14000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 20000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 24000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 25000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceInc

In [31]:
# fixing detailedType
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'flat', 'subTypology': 'standardFlat'}" if x == "{'typology': 'flat'}" else x)
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'chalet', 'subTypology': 'standardChalet'}" if x == "{'typology': 'chalet'}" else x)

In [32]:
# Converting string of dicts --> dicts

from ast import literal_eval # Library for string evaluation

df['parkingSpace'] = df['parkingSpace'].apply(literal_eval)
df['highlight'] = df['highlight'].apply(literal_eval)
df['detailedType'] = df['detailedType'].apply(literal_eval)

In [33]:
df = pd.concat([df,
           pd.json_normalize(df['parkingSpace']),
           pd.json_normalize(df['highlight']),
           pd.json_normalize(df['detailedType'])],
          axis = 1)

In [34]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,parkingSpace,priceByArea,detailedType,highlight,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,269000.0,flat,89.0,True,2,2,Calle Halcon,Madrid,Madrid,40.386730,...,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",3022.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},True,True,NaN,None,flat,standardFlat
1,180000.0,flat,63.0,True,3,1,Calle Carlos Paíno,Madrid,Madrid,40.386513,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",2857.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2,267000.0,flat,84.0,True,3,2,Calle del Espinar,Madrid,Madrid,40.386024,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",3179.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
3,104500.0,flat,58.0,False,2,1,barrio Vista Alegre,Madrid,Madrid,40.386165,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",1802.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
4,240000.0,flat,91.0,True,3,2,Calle de Antolina Merino s/n,Madrid,Madrid,40.386396,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",2637.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,199000.0,flat,47.0,False,1,1,"Calle Lavapiés, 52",Madrid,Madrid,40.409122,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4234.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat
2496,199000.0,flat,47.0,False,1,1,"Calle Lavapiés, 52",Madrid,Madrid,40.409122,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4234.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2497,160000.0,flat,83.0,True,2,1,Calle santa lucia,Madrid,Leganés,40.357973,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",1928.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2498,360000.0,flat,51.0,True,1,1,barrio Sol,Madrid,Madrid,40.414290,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",7059.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat


In [35]:
del df['parkingSpace']
del df['highlight']
del df['detailedType']

In [36]:
df.columns

Index(['price', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms',
       'address', 'province', 'municipality', 'latitude', 'longitude',
       'status', 'newDevelopment', 'hasLift', 'priceByArea', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'groupDescription', 'typology', 'subTypology'],
      dtype='object')

In [37]:
df2 = pd.read_csv('recleaned_df.csv')

In [38]:
df2.columns

Index(['price', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms',
       'address', 'province', 'municipality', 'latitude', 'longitude',
       'status', 'newDevelopment', 'hasLift', 'priceByArea', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'groupDescription', 'typology', 'subTypology'],
      dtype='object')

In [39]:
df.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [40]:
combined = pd.concat([df, df2], axis = 0)
combined.shape

(17205, 21)

In [41]:
combined = combined.drop_duplicates()
combined

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,status,newDevelopment,hasLift,priceByArea,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,269000.0,flat,89.0,True,2,2,Calle Halcon,Madrid,Madrid,40.386730,...,good,False,True,3022.0,True,True,NaN,None,flat,standardFlat
1,180000.0,flat,63.0,True,3,1,Calle Carlos Paíno,Madrid,Madrid,40.386513,...,good,False,False,2857.0,False,False,NaN,None,flat,standardFlat
2,267000.0,flat,84.0,True,3,2,Calle del Espinar,Madrid,Madrid,40.386024,...,good,False,True,3179.0,False,False,NaN,None,flat,standardFlat
3,104500.0,flat,58.0,False,2,1,barrio Vista Alegre,Madrid,Madrid,40.386165,...,good,False,False,1802.0,False,False,NaN,None,flat,standardFlat
4,240000.0,flat,91.0,True,3,2,Calle de Antolina Merino s/n,Madrid,Madrid,40.386396,...,good,False,True,2637.0,False,False,NaN,None,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14700,830000.0,flat,175.0,True,5,4,Calle de Hilarión Eslava,Madrid,Madrid,40.435605,...,good,False,True,4743.0,False,False,NaN,NaN,flat,standardFlat
14701,889000.0,flat,118.0,True,2,2,Mesonero Romanos,Madrid,Madrid,40.420548,...,good,False,True,7534.0,False,False,NaN,Destacado,flat,standardFlat
14702,4200000.0,flat,471.0,True,6,5,Calle de Velázquez,Madrid,Madrid,40.425091,...,good,False,True,8917.0,True,True,NaN,Destacado,flat,standardFlat
14703,1050000.0,duplex,233.0,True,3,4,barrio Nuevos Ministerios-Ríos Rosas,Madrid,Madrid,40.444252,...,good,False,True,4506.0,False,False,NaN,NaN,flat,duplex


In [29]:
combined.to_csv('recleaned_df.csv', index = False)

# Recleaned

In [30]:
import pandas as pd

In [34]:
pd.read_csv('recleaned_df.csv')

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,status,newDevelopment,hasLift,priceByArea,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,295000.0,flat,67.0,True,1,1,Calle de los Misterios,Madrid,Madrid,40.441226,...,good,False,False,4403.0,False,False,NaN,NaN,flat,standardFlat
1,710000.0,chalet,172.0,False,2,2,barrio Concepción,Madrid,Madrid,40.441675,...,renew,False,False,4128.0,True,True,NaN,NaN,chalet,semidetachedHouse
2,800000.0,flat,101.0,True,2,2,barrio Concepción,Madrid,Madrid,40.441166,...,newdevelopment,True,True,7921.0,True,True,NaN,NaN,flat,standardFlat
3,995000.0,flat,190.0,True,3,2,barrio Concepción,Madrid,Madrid,40.441759,...,newdevelopment,True,True,5237.0,True,True,NaN,NaN,flat,standardFlat
4,1020000.0,flat,177.0,True,3,2,barrio Concepción,Madrid,Madrid,40.440526,...,newdevelopment,True,True,5763.0,True,True,NaN,NaN,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14700,830000.0,flat,175.0,True,5,4,Calle de Hilarión Eslava,Madrid,Madrid,40.435605,...,good,False,True,4743.0,False,False,NaN,NaN,flat,standardFlat
14701,889000.0,flat,118.0,True,2,2,Mesonero Romanos,Madrid,Madrid,40.420548,...,good,False,True,7534.0,False,False,NaN,Destacado,flat,standardFlat
14702,4200000.0,flat,471.0,True,6,5,Calle de Velázquez,Madrid,Madrid,40.425091,...,good,False,True,8917.0,True,True,NaN,Destacado,flat,standardFlat
14703,1050000.0,duplex,233.0,True,3,4,barrio Nuevos Ministerios-Ríos Rosas,Madrid,Madrid,40.444252,...,good,False,True,4506.0,False,False,NaN,NaN,flat,duplex
